In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define transformations: convert to tensor and normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # mean and std for MNIST
])

In [3]:
# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:04<00:00, 2.39MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 90.5kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 846kB/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [4]:
# Define a simple feedforward neural network
class SimpleFFNN(nn.Module):
    def __init__(self):
        super(SimpleFFNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)  # hidden layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)     # output layer

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# Instantiate the network, define loss and optimizer
model = SimpleFFNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# Training loop
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}]  Loss: {loss.item():.6f}')

In [7]:
# Testing loop
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy*100:.2f}%)')

In [8]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleFFNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [9]:
# Train for 5 epochs
for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion)

Train Epoch: 1 [0/60000]  Loss: 2.379567
Train Epoch: 1 [6400/60000]  Loss: 0.294139
Train Epoch: 1 [12800/60000]  Loss: 0.277374
Train Epoch: 1 [19200/60000]  Loss: 0.244020
Train Epoch: 1 [25600/60000]  Loss: 0.276662
Train Epoch: 1 [32000/60000]  Loss: 0.192224
Train Epoch: 1 [38400/60000]  Loss: 0.216590
Train Epoch: 1 [44800/60000]  Loss: 0.248662
Train Epoch: 1 [51200/60000]  Loss: 0.270294
Train Epoch: 1 [57600/60000]  Loss: 0.137064
Test set: Average loss: 0.1358, Accuracy: 9582/10000 (95.82%)
Train Epoch: 2 [0/60000]  Loss: 0.201892
Train Epoch: 2 [6400/60000]  Loss: 0.069879
Train Epoch: 2 [12800/60000]  Loss: 0.074729
Train Epoch: 2 [19200/60000]  Loss: 0.051494
Train Epoch: 2 [25600/60000]  Loss: 0.053794
Train Epoch: 2 [32000/60000]  Loss: 0.219389
Train Epoch: 2 [38400/60000]  Loss: 0.081733
Train Epoch: 2 [44800/60000]  Loss: 0.069754
Train Epoch: 2 [51200/60000]  Loss: 0.117569
Train Epoch: 2 [57600/60000]  Loss: 0.110541
Test set: Average loss: 0.0946, Accuracy: 9711/1